In [39]:
# 1. Sum total of all aWattar prices over periods from 00:00 to 24:00=00:00 …. V1 = Sum (Awattar)
# 2. Calculation of average aWattar prices of this period (T) in hours (days * 24) … V2 = V1/T

# 3. Calculation of energy consumed over period T … V3 = Sum (Energy)

# 4. Calculation of aWattar reference costs … Cref = V2 x V3 / 1000
# 5. Calculation of costs for every period of energy of hour x awattar of hour … c = e * a /1000
# 6. Calculation of sum of hourly energy costs … C = Sum (c)
# 7. Calculation of average savings in € … S€ = Cref – C
# 8. Calculation of average savings in % … S% = (Cref – C) / Cref *100

import sys 
sys.path.append('/home/pi/smart_plug/')
import requests
from datetime import datetime, timedelta
import pytz
import pandas as pd
import json
import math
from main_services.common_utils import common_utils 

auth_url = common_utils.static_smart_meter_service_link+'orchestration/Authentication/Login'
auth_payload = {"user": common_utils.static_smart_meter_username, "pwd": common_utils.static_smart_meter_password}
auth_response = None

def avgAwattarPricesByDates_V2(total_marketprice, T):return roundUpCeil(total_marketprice/T)


def sumEnegrySmartMeterByDates_V3(): 
    auth_cookie, auth_xsrf_token, nsc_wt = None, None, None
    auth_response = requests.post(auth_url, json=auth_payload)
    if auth_response.status_code == 200:
        print("Smart meter - Authentication successful...")
        auth_cookie = auth_response.cookies['__Host-go4DavidSecurityToken']
        auth_xsrf_token = auth_response.cookies['XSRF-Token']
        nsc_wt = auth_response.cookies['NSC_WT_TWYUXFCQ-TTM']
    else:
        raise Exception("Authentication failed with status code: Smartmeter_service line 82",
                        auth_response.status_code)

    start_date = "2023-12-20 00:00:00"
    end_date = "2023-12-20 23:59:59"

    if (start_date is None) or (end_date is None):
            raise Exception("Please provide a 'start_date' and 'end_date' in the format: Eg:'2023-8-24'")

    if auth_cookie and auth_xsrf_token is None:
        print("Authentication failed with status code:", auth_response.status_code)
    
    headers = {
        'Cookie': f'__Host-go4DavidSecurityToken={auth_cookie};',
        'Cookie': f'XSRF-Token={auth_xsrf_token};',
        'Cookie': f'NSC_WT_TWYUXFCQ-TTM={auth_xsrf_token};',
    }

    data_url = f"{common_utils.static_smart_meter_service_link}orchestration/ConsumptionRecord/Week?meterId={common_utils.static_smart_meter_meter_id}&startDate={start_date}&endDate={end_date}"
    data_response = requests.get(data_url, headers=headers)
    if data_response.status_code == 200:
        data_response.json()
    else:
        print("Data request failed with status code:", 'DATA FAILURE')
        raise Exception("Data request failed with status code: Smartmeter_service2",
                        data_response.status_code)

    if data_response is not None:
        data_response = pd.DataFrame(data_response)[['meteredValues', 'peakDemandTimes']]
        print(data_response)
        return data_response.to_json()
    else:
        print("No data found")

def awattarReferenceCost_CREF(): 
    
    return 

def sumAwattarPricesByDates_V1():
    awattar_json_url = "https://api.awattar.at/v1/marketdata?start={}&end={}"

    timezone = 'Europe/Vienna'  # Replace with your desired timezone

    # Example usage
    from_timestamp = 1640179200000  # Replace with the actual timestamp
    to_timestamp = 1640438399999    # Replace with the actual timestamp

    start_of_day, end_of_day, number_of_date = get_start_and_end_of_day(from_timestamp, to_timestamp, timezone)

    ## print(f"Awattar After Start of the day: {start_of_day}")
    ## print(f"Awattar After End of the day: {end_of_day}")

    # Get Awattar Data
    json_url = awattar_json_url.format(start_of_day, end_of_day)
    
    awattar_json_response = requests.get(json_url).json()
    awattar_json_response = pd.json_normalize(awattar_json_response['data'])
    awattar_json_response['start_timestamp'] = pd.to_datetime(awattar_json_response['start_timestamp'], unit='ms') + pd.Timedelta(hours=1)
    awattar_json_response['end_timestamp'] = pd.to_datetime(awattar_json_response['end_timestamp'], unit='ms') + pd.Timedelta(hours=1)
    ## print(awattar_json_response)

    # Calculate the sum of marketprice
    total_marketprice = awattar_json_response['marketprice'].sum()

    print("V1:", roundUpCeil(total_marketprice))


    T = number_of_date * 24

    print("V2:", avgAwattarPricesByDates_V2(total_marketprice, T))

"""
Other utility methods to calculate the reports
22.12.23
"""

def get_start_and_end_of_day(from_timestamp, to_timestamp, timezone='CET'):
    # Convert timestamps to datetime objects
    from_date = datetime.utcfromtimestamp(from_timestamp / 1000)
    to_date = datetime.utcfromtimestamp(to_timestamp / 1000)

    # Set the timezone to UTC
    from_date_utc = pytz.timezone('UTC').localize(from_date)
    to_date_utc = pytz.timezone('UTC').localize(to_date)

    # Convert UTC to the specified local time zone
    from_date_local = from_date_utc.astimezone(pytz.timezone(timezone))
    to_date_local = to_date_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day for both from and to dates
    start_of_day_from = from_date_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day_from = start_of_day_from.replace(hour=23, minute=59, second=59, microsecond=999999)

    start_of_day_to = to_date_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day_to = start_of_day_to.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Get the start of the next day for both from and to dates
    start_of_next_day_from = start_of_day_from + timedelta(days=1)
    start_of_next_day_to = start_of_day_to + timedelta(days=1)

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp_from = int(start_of_day_from.timestamp()) * 1000
    end_timestamp_from = int(start_of_next_day_from.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    start_timestamp_to = int(start_of_day_to.timestamp()) * 1000
    end_timestamp_to = int(start_of_next_day_to.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    ##print(f"Start of the day for 'From': {start_of_day_from}")
    ##print(f"End of the day for 'From' (including next start): {end_of_day_from}")

    ##print(f"Start of the day for 'To': {start_of_day_to}")
    ##print(f"End of the day for 'To' (including next start): {end_of_day_to}")
    
    number_of_date = numberOfDaysFromDates(start_timestamp_from, end_timestamp_to)

    return start_timestamp_from, end_timestamp_to, number_of_date

def numberOfDaysFromDates(start_timestamp_from, end_timestamp_to):
    
    # Convert timestamps to datetime objects
    start_date = datetime.utcfromtimestamp(start_timestamp_from / 1000)
    end_date = datetime.utcfromtimestamp(end_timestamp_to / 1000)

    # Calculate the difference in days
    days_difference = (end_date - start_date).days

    ##print(f"Number of days between the two dates: {days_difference} days")
    return days_difference

def roundUpCeil(value):
    return math.ceil(value)

## init method
## sumAwattarPricesByDates_V1()
sumEnegrySmartMeterByDates_V3()


Smart meter - Authentication successful...
Data request failed with status code: DATA FAILURE


Exception: ('Data request failed with status code: Smartmeter_service2', 401)